In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from pathlib import Path
from sklearn.preprocessing import MinMaxScaler

from models.StackedAutoEncoder import SimpleAutoEncoder
from models.LSTM import LSTM, GRU

In [6]:
X_train = torch.tensor(np.load(Path("model_data/x_train.npy")), dtype=torch.float32)
X_val = torch.tensor(np.load(Path("model_data/x_val.npy")), dtype=torch.float32)
X_test = torch.tensor(np.load(Path("model_data/x_test.npy")), dtype=torch.float32)
Y_train = torch.tensor(np.load(Path("model_data/y_train.npy")), dtype=torch.float32)
Y_val = torch.tensor(np.load(Path("model_data/y_val.npy")), dtype=torch.float32)
Y_test = torch.tensor(np.load(Path("model_data/y_test.npy")), dtype=torch.float32)
X_train.shape

torch.Size([482, 19, 21])

In [2]:
input_size = 21
hidden_size = 5
num_layers = 4
output_size = 1
dropout = 0.2
batch_size = 60
num_epochs = 5_000
lr = 0.01

In [ ]:
model = SimpleAutoEncoder(input_size=input_size, hidden_size=5, )
model.fit(X_train, epochs=5_000, lr=0.05, VERBOSE=False)

x_train_sae = model.forward(X_train)
x_val_sae = model.forward(X_val)
x_test_sae = model.forward(X_test)
x_train_sae.shape

In [3]:
np.save(Path("model_data/x_train_sae.npy"), x_train_sae.detach().numpy())
np.save(Path("model_data/x_val_sae.npy"), x_val_sae.detach().numpy())
np.save(Path("model_data/x_test_sae.npy"), x_test_sae.detach().numpy())

NameError: name 'x_train_sae' is not defined

In [7]:
X_train = torch.tensor(np.load(Path("model_data/x_train_sae.npy")), dtype=torch.float32)
X_val = torch.tensor(np.load(Path("model_data/x_val_sae.npy")), dtype=torch.float32)
X_test = torch.tensor(np.load(Path("model_data/x_test_sae.npy")), dtype=torch.float32)
X_train.shape

torch.Size([482, 19, 21])

In [8]:
model = LSTM(input_size=21, hidden_size=hidden_size, num_layers=num_layers, output_size=output_size, dropout=dropout)
# model.fit(x_train, y_train)

criterion = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=lr)
train_dataset = TensorDataset(X_train, Y_train)
train_dataset = train_dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val, Y_val)
val_dataset = val_dataset
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

train_hist = []
val_hist = []

for epoch in range(num_epochs):
    total_loss = 0.0
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.float()
        predictions = model(batch_X.float())

        predictions = predictions.float()  
        batch_y = batch_y.float()  

        loss = criterion(predictions, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    optimizer.step()
    average_loss = total_loss/len(train_loader)
    train_hist.append(average_loss)

    model.eval()
    with torch.no_grad():
        total_val_loss = 0.0

        for batch_X_val, batch_y_val in val_loader:
            predictions_val = model(batch_X_val.float())

            lossmsev = criterion(predictions_val,batch_y_val.float())
            val_loss = lossmsev
            total_val_loss += val_loss.item()

        average_val_loss = total_val_loss / len(val_loader)
        val_hist.append(average_val_loss)

    if (epoch + 1) % 1 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}] - Training Loss: {average_loss:.3f}, MSE Loss: {loss:.6f}, Val Loss: {average_val_loss:.6f}, Learning Rate: {optimizer.param_groups[0]["lr"]}')

/Users/christian/anaconda3/envs/a3/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([60, 21])) that is different to the input size (torch.Size([60, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/christian/anaconda3/envs/a3/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2, 21])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/christian/anaconda3/envs/a3/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([41, 21])) that is different to the input size (torch.Size([41, 1])). This will likely lead to incorrect results due to 

Epoch [1/5000] - Training Loss: 0.172, MSE Loss: 0.093236, Val Loss: 0.086909, Learning Rate: 0.01
Epoch [2/5000] - Training Loss: 0.080, MSE Loss: 0.060935, Val Loss: 0.085498, Learning Rate: 0.01
Epoch [3/5000] - Training Loss: 0.065, MSE Loss: 0.058169, Val Loss: 0.108156, Learning Rate: 0.01
Epoch [4/5000] - Training Loss: 0.068, MSE Loss: 0.079386, Val Loss: 0.087516, Learning Rate: 0.01
Epoch [5/5000] - Training Loss: 0.070, MSE Loss: 0.081674, Val Loss: 0.085993, Learning Rate: 0.01
Epoch [6/5000] - Training Loss: 0.070, MSE Loss: 0.115657, Val Loss: 0.094201, Learning Rate: 0.01
Epoch [7/5000] - Training Loss: 0.066, MSE Loss: 0.075653, Val Loss: 0.087739, Learning Rate: 0.01
Epoch [8/5000] - Training Loss: 0.063, MSE Loss: 0.060226, Val Loss: 0.098932, Learning Rate: 0.01
Epoch [9/5000] - Training Loss: 0.063, MSE Loss: 0.036778, Val Loss: 0.095887, Learning Rate: 0.01
Epoch [10/5000] - Training Loss: 0.063, MSE Loss: 0.060098, Val Loss: 0.087186, Learning Rate: 0.01
Epoch [11

In [9]:
model = GRU(input_size=21, hidden_size=hidden_size, num_layers=num_layers, output_size=output_size, dropout=dropout)
# model.fit(X_train, Y_train, epochs=1_000)

criterion = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=lr)
train_dataset = TensorDataset(X_train, Y_train)
train_dataset = train_dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val, Y_val)
val_dataset = val_dataset
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

train_hist = []
val_hist = []

for epoch in range(num_epochs):
    total_loss = 0.0
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.float()
        predictions = model(batch_X.float())

        predictions = predictions.float()  
        batch_y = batch_y.float()  

        loss = criterion(predictions, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    optimizer.step()
    average_loss = total_loss/len(train_loader)
    train_hist.append(average_loss)

    model.eval()
    with torch.no_grad():
        total_val_loss = 0.0

        for batch_X_val, batch_y_val in val_loader:
            predictions_val = model(batch_X_val.float())

            lossmsev = criterion(predictions_val,batch_y_val.float())
            val_loss = lossmsev
            total_val_loss += val_loss.item()

        average_val_loss = total_val_loss / len(val_loader)
        val_hist.append(average_val_loss)

    if (epoch + 1) % 1 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}] - Training Loss: {average_loss:.3f}, MSE Loss: {loss:.6f}, Val Loss: {average_val_loss:.6f}, Learning Rate: {optimizer.param_groups[0]["lr"]}')

Epoch [1/5000] - Training Loss: 0.161, MSE Loss: 0.137263, Val Loss: 0.095086, Learning Rate: 0.01
Epoch [2/5000] - Training Loss: 0.068, MSE Loss: 0.060704, Val Loss: 0.119257, Learning Rate: 0.01
Epoch [3/5000] - Training Loss: 0.073, MSE Loss: 0.090621, Val Loss: 0.092183, Learning Rate: 0.01
Epoch [4/5000] - Training Loss: 0.064, MSE Loss: 0.071821, Val Loss: 0.091689, Learning Rate: 0.01
Epoch [5/5000] - Training Loss: 0.065, MSE Loss: 0.086888, Val Loss: 0.091803, Learning Rate: 0.01
Epoch [6/5000] - Training Loss: 0.064, MSE Loss: 0.065685, Val Loss: 0.094952, Learning Rate: 0.01
Epoch [7/5000] - Training Loss: 0.063, MSE Loss: 0.079346, Val Loss: 0.087875, Learning Rate: 0.01
Epoch [8/5000] - Training Loss: 0.057, MSE Loss: 0.055590, Val Loss: 0.087928, Learning Rate: 0.01
Epoch [9/5000] - Training Loss: 0.049, MSE Loss: 0.042494, Val Loss: 0.089917, Learning Rate: 0.01
Epoch [10/5000] - Training Loss: 0.056, MSE Loss: 0.065514, Val Loss: 0.083586, Learning Rate: 0.01
Epoch [11

KeyboardInterrupt: 